In [45]:
import json
import os
import random
import fnmatch
import openpyxl

from urllib.request import urlopen
import urllib

import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import re


%matplotlib inline

In [46]:
sns.set_style("darkgrid")

In [47]:
def import_json_to_dict(url) :
    response = urllib.request.urlopen(url)
    my_dict = json.loads(response.read())
    return my_dict

In [48]:
def clean_str(s):
    d = {
        "’": "'",
        "\xa0": " ",
        "/":","
        }
    for x in d:
        s = s.replace(x, d[x]).strip()
    return s

In [49]:
month_dict = {
    "January" : "Janvier",
"February": "Février",
"March": "Mars",
"April" : "Avril",
"May" : "Mai",
"June" : "Juin",
"July" : "Juillet",
"August" : "Août",
"September":"Septembre",
"October":"Octobre",
"November":"Novembre",
"December":"Décembre"}

In [50]:
data_dir_path = './data/'

In [51]:
#Taxonomies

taxo_dep_url = 'https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/taxonomies/departements-minify.json'
taxo_reg_url = 'https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/taxonomies/regions-minify.json'

In [52]:
mailles_list = ['national', 'regional', 'departemental']

In [53]:
propilot_path = os.path.join("data")

In [54]:
!ls $propilot_path

dim_activity.csv      dim_properties.csv    dim_top_levels.csv
dim_effects.csv       dim_snapshots.csv     dim_tree_nodes.csv
dim_maturities.csv    dim_states_pp_ch.csv  fact_financials_enr_pp_ch.csv
dim_period_pp_ch.csv  dim_structures.csv    fact_property_values.csv


In [55]:
df_dict = {}
file_list = [
"dim_activity",
"dim_tree_nodes",
"dim_top_levels",
"dim_maturities",
"dim_period",
"dim_snapshots",
"dim_effects",
"dim_properties",
"dim_states",
"dim_structures",
"fact_financials",
"fact_property_values"
]

In [56]:
for file in file_list:
    for file_csv in os.listdir(propilot_path):
        if fnmatch.fnmatch(file_csv, file + "*.csv"):
            print("File loaded : ", file_csv)
            df_dict[file] = pd.read_csv(os.path.join(propilot_path, file_csv), sep=";")

File loaded :  dim_activity.csv
File loaded :  dim_tree_nodes.csv
File loaded :  dim_top_levels.csv
File loaded :  dim_maturities.csv
File loaded :  dim_period_pp_ch.csv
File loaded :  dim_snapshots.csv
File loaded :  dim_effects.csv
File loaded :  dim_properties.csv
File loaded :  dim_states_pp_ch.csv
File loaded :  dim_structures.csv
File loaded :  fact_financials_enr_pp_ch.csv
File loaded :  fact_property_values.csv


In [150]:
df = (df_dict["fact_financials"]
      .merge(df_dict["dim_tree_nodes"], left_on="tree_node_id", right_on="tree_node_id")
      .merge(df_dict["dim_effects"], left_on="effect_id", right_on="effect_id")
      .merge(df_dict["dim_states"], left_on="state_id", right_on="state_id")
      .merge(df_dict["dim_period"], left_on="period_id", right_on="period_id")
      .merge(df_dict["dim_structures"], left_on="structure_id", right_on="structure_id"))

In [151]:
def extract_dep_code(expr):
    nums = re.findall(r'\d+', expr)
    #assert len(nums) <= 1, f"{nums}"
    return nums[0].zfill(2) if len(nums) > 0 else None

df['dep_code'] = df['tree_node_code'].apply(lambda x: extract_dep_code(x))

In [154]:
cols = ["tree_node_name", "structure_name", "effect_id", "period_date", "period_month_tri", "period_month_year", "financials_cumulated_amount", "dep_code"]
df=df[cols]
df.rename(columns={"period_month_year":"Date", "financials_cumulated_amount":"valeur"}, inplace=True)

In [155]:
forbidden_period_value = ["Y", "Total"]

In [156]:
df_dep = df.loc[
    (df.structure_name == "Département") & 
    (~df.period_month_tri.isin(forbidden_period_value))].copy()

In [157]:
df_dep["departement"] = df_dep["tree_node_name"].apply(lambda x: x.split("/")[0].strip())
df_dep["mesure"] = df_dep["tree_node_name"].apply(lambda x: x.split("/")[1].strip())

# nettoyage de la colonne mesure, on enlève un point surnuméraire.
df_dep["mesure"].replace("\.", "", regex=True,inplace=True)
df_dep.mesure = df_dep.mesure.apply(lambda x: clean_str(x))

df_dep.rename(columns={"effect_id":"indicateur"}, inplace=True)
df_dep.indicateur = df_dep.indicateur.str.strip()
# traduit les mois dans la colonne date
df_dep.Date = df_dep.Date.replace(month_dict, regex=True)

df_dep.valeur.replace({",": "."}, regex=True, inplace=True)
df_dep.valeur = df_dep.valeur.astype(float)

In [158]:
df_dep["short_indic"] = df_dep.indicateur.apply(lambda x: x.split("-")[0].strip())
df_dep.short_indic = df_dep.short_indic.apply(lambda x: clean_str(x))

In [159]:
dict_indicateur = {'Nombre de repas servis dans les restaurants universitaires au tarif à 1€' : "Nombre de repas servis",
 'Montant cumulé de l’investissement total ainsi déclenché' : "Montant cumumé de l'investissement total",
 'Nombre d’entreprises bénéficiaires du dispositif' : "Nombre d'entreprises bénéficiaires",
 'Nombre de bâtiments Etat dont le marché de rénovation est notifié' : 'Nombre de bâtiments dont le marché de rénovation est notifié',
 'Nombre de projets incluant une transformation de la ligne de production pour réduire son impact environnemental' : 'Nombre de projets',
 'Nombre d\'exploitations certifiées "haute valeur environnementale"' : 'Nombre d’exploitations certifiées',
 'Emissions de gaz à effet de serre évitées sur la durée de vie des équipements' : 'Emissions de gaz à effet de serre évitées',
 'Nombre de bonus octroyés à des véhicules électriques et hybrides rechargeables' : 'Nombre de bonus octroyés à des véhicules électriques',
 "Quantité de matières plastiques évitées ou dont le recyclage ou l'intégration a été soutenue" : 'Quantité de matières plastiques évitées',
 'Montant total des travaux associés aux dossiers validés' : 'Montant total des travaux',
 'Nombre de nouveaux projets (nouvelle ligne, extension de ligne et pôle)' : 'Nombre de nouveaux projets',
 'Montant de l’investissement total déclenché' : 'Montant de l’investissement total',
 'Nombre de projets de tourisme durable financés' : 'Nombre de projets de tourisme durable financés',
 'Nombre de projets de rénovation de cathédrales et de monuments nationaux initiés' : 'Nombre de projets de rénovation',
 'Montant total investi pour la rénovation de monuments historiques appartenant aux collectivités territoriales' :'Montant total investi pour la rénovation',
 'Nombre de projets de rénovation de monuments historiques appartenant aux collectivités territoriales bénéficiaires initiés' : 'Nombre de projets de rénovation',
 "Nombre de contrats de professionnalisation bénéficiaires de l'aide exceptionnelle" : 'Nombre de contrats de professionnalisation',
 "Nombre de contrats d'apprentissage bénéficiaires de l'aide exceptionnelle" : 'Nombre de contrats d’apprentissage',
 "Nombre d'aides à l'embauche des travailleurs handicapés": "Nombre d'aides à l'embauche des travailleurs handicapés",
 'Nombre de projets locaux soutenus  (rénovation, extension, création de lignes' :'Nombre de projets locaux soutenus'
                  }

In [160]:
df_dep.short_indic = df_dep.short_indic.apply(lambda x: dict_indicateur[x] if x in dict_indicateur else x)

In [161]:
[x for x in list(df_dep.short_indic.unique()) if len(x) >40]

["Montant cumulé de l'investissement total ainsi déclenché",
 "Nombre d'entreprises bénéficiaires du dispositif",
 'Nombre de bâtiments dont le marché de rénovation est notifié',
 'Emissions de gaz à effet de serre évitées',
 'Nombre de bonus octroyés à des véhicules électriques',
 "Montant de l'investissement total déclenché",
 'Nombre de projets de tourisme durable financés',
 'Nombre de contrats de professionnalisation',
 "Nombre d'aides à l'embauche des travailleurs handicapés"]

In [162]:
dict_mesures  = {
 "Appels à projets pour les projets d'efficacité énergétique de grande ampleur et appels à manifestation d'intérêt sur les projets qui proposent une évolution profonde des procédés industriels": "AAP et AMI Efficacité énergétique",
 "CIE jeunes" : "Contrats Initiatives Emploi (CIE) Jeunes",
 'France Num' : 'France Num : aide à la numérisation des TPE,PME,ETI',
 'Guichet efficacité énergétique dans industrie' : 'Guichet efficacité énergétique',
 "Modernisation de la filière (BPI)" : "AAP industrie : Modernisation des filières auto et aéro",
 "PEC jeunes": "Parcours emploi compétences (PEC) Jeunes",
 'Relocalisation : soutien aux projets industriels dans les territoires' : 'AAP Industrie : Soutien aux projets industriels territoires',
 'Relocalisation : sécurisation des approvisionnements critiques' : 'AAP Industrie : Sécurisation approvisionnements critiques',
 'Renforcement des subventions de Business France (chèque export, chèque VIE)' : 'Renforcement subventions Business France',
 "Soutien à la modernisation industrielle et renforcement des compétences dans la filière nucléaire" : "AAP industrie : modernisation industrielle et renforcement des compétences dans la filière nucléaire",
 'Soutien à la recherche aéronautique civil' :'Soutien recherche aéronautique civil',
 'Rénovation bâtiments Etats' : 'Rénovation des bâtiments Etats (marchés notifiés)'
}

In [163]:
df_dep["short_mesure"] = df_dep.mesure.apply(lambda x: dict_mesures[x] if x in dict_mesures else x)

In [164]:
taxo_dep_url = 'https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/taxonomies/departements-minify.json'
taxo_reg_url = 'https://raw.githubusercontent.com/etalab/dashboard-aides-entreprises/master/backend/json/taxonomies/regions-minify.json'
taxo_dep_df = pd.DataFrame(import_json_to_dict(taxo_dep_url))
dep_list = list(taxo_dep_df['dep'])
print('{} departements.'.format(len(dep_list)))

taxo_reg_df = pd.DataFrame(import_json_to_dict(taxo_reg_url))
reg_list = list(taxo_reg_df['reg'])
print('{} regions.'.format(len(reg_list)))

101 departements.
18 regions.


In [136]:
taxo_dep_df

,cheflieu,dep,libelle,ncc,nccenr,reg,tncc
0,01053,01,Ain,AIN,Ain,84,5
1,02408,02,Aisne,AISNE,Aisne,32,5
2,03190,03,Allier,ALLIER,Allier,84,5
3,04070,04,Alpes-de-Haute-Provence,ALPES DE HAUTE PROVENCE,Alpes-de-Haute-Provence,93,4
4,05061,05,Hautes-Alpes,HAUTES ALPES,Hautes-Alpes,93,4
...,...,...,...,...,...,...,...
96,97105,971,Guadeloupe,GUADELOUPE,Guadeloupe,01,3
97,97209,972,Martinique,MARTINIQUE,Martinique,02,3
98,97302,973,Guyane,GUYANE,Guyane,03,3
99,97411,974,La Réunion,LA REUNION,La Réunion,04,0


In [168]:
df_dep_enr = df_dep.merge(taxo_dep_df[["dep", "reg", "libelle"]], 
                          how="left", left_on="dep_code", right_on="dep")\
    .merge(taxo_reg_df[["reg", "nccenr"]], how="left", left_on="reg", right_on="reg")
df_dep_enr.drop(columns=["tree_node_name", "structure_name"], inplace=True)
df_dep_enr = df_dep_enr.loc[~df_dep_enr.dep.isna(), :]
df_dep_enr.rename(columns={"nccenr":"region"}, inplace=True)
df_dep_enr.shape

(13530, 14)

In [70]:
df_dep_enr.to_csv("pp_dep.csv", sep=";")

In [170]:
df_dep_enr

,indicateur,period_date,period_month_tri,Date,valeur,dep_code,departement,mesure,short_indic,short_mesure,dep,reg,libelle,region
0,Nombre de repas servis dans les restaurants un...,2021-01-31T00:00:00.0000000,Jan,Janvier 2021,251913.0,14,Calvados,Repas à 1 € dans les restaurants universitaires,Nombre de repas servis,Repas à 1 € dans les restaurants universitaires,14,28,Calvados,Normandie
1,Nombre de repas servis dans les restaurants un...,2021-01-31T00:00:00.0000000,Jan,Janvier 2021,240329.0,44,Loire-Atlantique,Repas à 1 € dans les restaurants universitaires,Nombre de repas servis,Repas à 1 € dans les restaurants universitaires,44,52,Loire-Atlantique,Pays de la Loire
2,Nombre de repas servis dans les restaurants un...,2021-01-31T00:00:00.0000000,Jan,Janvier 2021,327402.0,33,Gironde,Repas à 1 € dans les restaurants universitaires,Nombre de repas servis,Repas à 1 € dans les restaurants universitaires,33,75,Gironde,Nouvelle-Aquitaine
3,Nombre de repas servis dans les restaurants un...,2021-01-31T00:00:00.0000000,Jan,Janvier 2021,332339.0,62,Pas-de-Calais,Repas à 1 € dans les restaurants universitaires,Nombre de repas servis,Repas à 1 € dans les restaurants universitaires,62,32,Pas-de-Calais,Hauts-de-France
4,Nombre de repas servis dans les restaurants un...,2021-01-31T00:00:00.0000000,Jan,Janvier 2021,421341.0,34,Hérault,Repas à 1 € dans les restaurants universitaires,Nombre de repas servis,Repas à 1 € dans les restaurants universitaires,34,76,Hérault,Occitanie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14415,Kilométrage d'aménagements cyclables - RPC,2021-03-31T00:00:00.0000000,Mar,Mars 2021,0.0,49,Maine-et-Loire,DRI : Aménagement cyclable et transports en co...,Kilométrage d'aménagements cyclables,DRI : Aménagement cyclable et transports en co...,49,52,Maine-et-Loire,Pays de la Loire
14416,Kilométrage d'aménagements cyclables - RPC,2021-03-31T00:00:00.0000000,Mar,Mars 2021,0.0,54,Meurthe-et-Moselle,DRI : Aménagement cyclable et transports en co...,Kilométrage d'aménagements cyclables,DRI : Aménagement cyclable et transports en co...,54,44,Meurthe-et-Moselle,Grand Est
14417,Kilométrage d'aménagements cyclables - RPC,2021-03-31T00:00:00.0000000,Mar,Mars 2021,0.0,02,Corse-du-Sud,DRI : Aménagement cyclable et transports en co...,Kilométrage d'aménagements cyclables,DRI : Aménagement cyclable et transports en co...,02,32,Aisne,Hauts-de-France
14418,Kilométrage d'aménagements cyclables - RPC,2021-03-31T00:00:00.0000000,Mar,Mars 2021,0.0,44,Loire-Atlantique,DRI : Aménagement cyclable et transports en co...,Kilométrage d'aménagements cyclables,DRI : Aménagement cyclable et transports en co...,44,52,Loire-Atlantique,Pays de la Loire


In [167]:
df_dep_enr.shape

(13395, 13)